<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

In [1]:
import re, json, time
from selenium import webdriver
import os

# import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# from webdriver_manager.chrome import ChromeDriverManager
from parsel import Selector

import pandas as pd
import numpy as np


# Tratamento


In [2]:
with open("canais.txt", "r") as f:
    # read each line as a item in a list and remove the \n
    canais = [line.strip() for line in f.readlines()]


def scroll_page(url):
    # service = Service(ChromeDriverManager().install())

    # options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--lang=en")
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    # )

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)

    time.sleep(3)
    selector = Selector(driver.page_source)
    driver.quit()

    return selector

In [3]:
import re


import locale

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

base_path = f"./data/{canais[0].split('@')[-1]}/"

print(base_path)

./data/syngentadigitalbrasil/


In [4]:
def trata_inscritos(col: pd.Series) -> pd.Series:
    col = col.split(" inscritos")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    if "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        return int(
            float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
        )

    return int(col.replace(" ", "").replace(",", "."))


def trata_visualizacoes(col):
    # treat the characters U+00a0

    # if its a numeric value return it
    if type(col) == int or type(col) == float:
        return col

    col = col.replace("\xa0", "")

    col = col.split(" visualizações")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    elif "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        return int(
            float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
        )

    try:
        return int(col.replace(" ", "").replace(".", "").replace(",", "."))
    except:
        return 0


def trata_comments(col):
    # print(col)
    # print(type(col))

    if type(col) == int or type(col) == float:
        return col

    value = 0

    try:
        value = int(col)
    except:
        pass

    if "Transcrição" in col:
        return 0

    col = col.replace("\xa0", "")

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    elif "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        try:
            value = int(
                float(col.replace("mi", "").replace(" ", "").replace(",", "."))
                * 1000000
            )
        except:
            value = 0

    return (
        int(col.replace(" ", "").replace(".", "").replace(",", "."))
        if value != 0
        else 0
    )


def trata_data(value):
    meses = {
        "jan": "01",
        "fev": "02",
        "mar": "03",
        "abr": "04",
        "mai": "05",
        "jun": "06",
        "jul": "07",
        "ago": "08",
        "set": "09",
        "out": "10",
        "nov": "11",
        "dez": "12",
    }
    series_data = re.sub(r"\. de", "", value)
    # print(series_data)
    series_data = re.search(r"(\d{1,2} de [a-zA-Z]+ \d{4})", series_data).group(0)
    # replace month
    for k, v in meses.items():
        series_data = series_data.replace(k, v)

    # print(series_data)
    series_data = pd.to_datetime(series_data, format="%d de %m %Y", errors="coerce")
    return series_data

trata videos

In [5]:
videos = pd.read_csv(base_path + "videos.csv")

display(videos)
display(videos.columns)

,title,views,likes,date,comments_amount,channel_name,subscribers
0,Caminho da inovação,visualizações,2,20 de nov. de 2023,Transcrição,Syngenta Digital Brasil,"3,26 mil inscritos"
1,Qual praga está colocando a sua lavoura em risco?,90 visualizações,3,19 de set. de 2023,Transcrição,Syngenta Digital Brasil,"3,26 mil inscritos"
2,"Plataforma Cropwise - Soluções conectadas, sem...",207 visualizações,1,8 de set. de 2023,Transcrição,Syngenta Digital Brasil,"3,26 mil inscritos"
3,Digital no suporte a tomada de decisão.,512 visualizações,16,29 de jun. de 2023,1,Syngenta Digital Brasil,"3,26 mil inscritos"
4,Cases de Reinvenção de 2022,170 visualizações,6,5 de jan. de 2023,Transcrição,Syngenta Digital Brasil,"3,26 mil inscritos"
5,"Do plantio à colheita, ao seu lado!",111 visualizações,15,9 de out. de 2023,5,Farmbox,553 inscritos
6,Conheça o Labs: Um mundo de possibilidades den...,143 visualizações,17,3 de ago. de 2023,4,Farmbox,553 inscritos
7,Agroimpar e Farmbox - História de sucesso,111 visualizações,12,27 de abr. de 2023,5,Farmbox,553 inscritos
8,NaN,NaN,0,NaN,NaN,Farmbox,553 inscritos
9,Alex Bruno - Fazenda Estrela Boscoli,420 visualizações,52,3 de jan. de 2023,3,Farmbox,553 inscritos


Index(['title', 'views', 'likes', 'date', 'comments_amount', 'channel_name',
       'subscribers'],
      dtype='object')

In [6]:
videos.dropna(subset="title").assign(
    # subscribers=lambda df_: df_["subscribers"].apply(trata_inscritos),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    date=lambda df_: df_["date"].apply(trata_data),
    # comments_amount = lambda df_: df_["comments_amount"].apply(trata_comments),
    category="RECENT",
).to_parquet(f"{base_path}videos.parquet")

# Trata Top videos

In [7]:
import pandas as pd


top_videos = pd.read_csv(base_path + "top_videos.csv")

display(top_videos.tail())

,title,views,likes,date,comments_amount,channel_name
20,História de sucesso #SouAegro - George Vital,175 mil visualizações,79,8 de mar. de 2023,2,Aegro
21,Como o grupo Nutriva Cereais maximizou sua ren...,115 mil visualizações,222,2 de jun. de 2023,3,Aegro
22,Conheça o aplicativo Aegro para gestão de faze...,106 mil visualizações,33,27 de out. de 2020,Conheça o aplicativo Aegro para gestão de faze...,Aegro
23,História de sucesso #SouAegro - Oberdan Ermita,104 mil visualizações,57,28 de mar. de 2023,Transcrição,Aegro
24,NaN,NaN,0,NaN,NaN,Aegro


In [10]:
top_videos.dropna().assign(
    date=lambda df_: df_["date"].apply(trata_data),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    # comments_amount = lambda df_: df_["comments_amount"].apply(trata_comments),

    category="TOP",
).to_parquet(base_path + "top_videos.parquet")

In [11]:
top_videos_silver = pd.read_parquet(base_path + "top_videos.parquet")

In [12]:
# # Trata lifetime


df_lifetime = pd.read_csv(f"{base_path}lifetime.csv")


df_lifetime.head()


def days_on_youtube(value) -> int:
    import datetime

    today = datetime.date.today()
    return (today - value.date()).days

In [13]:
df_lifetime

,channel_title,date_joined,total_views,total_subs,total_videos
0,Syngenta Digital Brasil,Inscreveu-se em 25 de jul. de 2014,833.066 visualizações,"3,26 mil inscritos",141
1,Farmbox,Inscreveu-se em 29 de jul. de 2019,27.752 visualizações,553 inscritos,32
2,xarvio Digital Farming Solutions,Inscreveu-se em 1 de abr. de 2019,19.743.313 visualizações,"1,99 mil inscritos",363
3,FieldView TV,Inscreveu-se em 9 de out. de 2015,10.345.865 visualizações,"10,6 mil inscritos",289
4,Aegro,Inscreveu-se em 27 de jun. de 2015,1.014.378 visualizações,"9,11 mil inscritos",169


In [14]:
df_lifetime = df_lifetime.assign(
    date_joined=lambda df_: df_["date_joined"].apply(trata_data),
    total_views=lambda df_: df_["total_views"].apply(trata_visualizacoes),
    total_videos=lambda d: d["total_videos"].apply(trata_visualizacoes),
    total_subs=lambda df_: df_["total_subs"].apply(trata_inscritos),
    days_since_joined=lambda df_: df_["date_joined"].apply(days_on_youtube),
)  # .to_csv(base_path + "lifetime_clean.csv", index=False)

In [15]:
lifetime_csv = pd.DataFrame()

lifetime_csv["Channel Id"] = np.arange(5)

lifetime_csv[" Channel Name"] = df_lifetime["channel_title"]
lifetime_csv[" Days on YouTube"] = df_lifetime["days_since_joined"]
lifetime_csv[" Total Views"] = df_lifetime["total_views"]
lifetime_csv[" Views per Day"] = (
    lifetime_csv[" Total Views"] / lifetime_csv[" Days on YouTube"]
)
lifetime_csv[" Total Subscribers"] = df_lifetime["total_subs"]
lifetime_csv[" Subscribers per Day"] = (
    lifetime_csv[" Total Subscribers"] / lifetime_csv[" Days on YouTube"]
)
lifetime_csv[" Total Uploads"] = df_lifetime["total_videos"]
lifetime_csv[" Uploads per Month"] = lifetime_csv[" Total Uploads"] / (
    lifetime_csv[" Days on YouTube"] / 30
)
lifetime_csv[""] = np.arange(5)

lifetime_csv.to_csv(base_path + "Lifetime_.csv", index=False, decimal=",")

In [16]:
# # join


videos = pd.read_parquet(base_path + "videos.parquet")
top_videos = pd.read_parquet(base_path + "top_videos.parquet")

In [17]:
videos = videos.assign(title_canal=lambda d: d["channel_name"] + " - " + d["title"])

top_videos = top_videos.assign(
    title_canal=lambda d: d["channel_name"] + " - " + d["title"]
)


# videos.sort_values("channel_name").to_csv(base_path + "videos_clean.csv", index=False, sep=";")
# top_videos.sort_values("channel_name").to_csv(base_path + "top_videos_clean.csv", index=False, sep=";")


# # join the two dataframes

# videos = pd.concat([videos, top_videos])


# videos_sorted = videos.sort_values("channel_name")

# videos_sorted["subscribers"] = videos_sorted["subscribers"].fillna(method="ffill")


# videos_sorted = videos_sorted.sort_values(["channel_name", "category", "date"])


# videos_sorted.to_parquet(base_path + "videos_sorted.parquet")


# (pd.read_parquet(base_path + "videos_sorted.parquet")
#  .assign(title_canal = lambda d: d['channel_name'] + ' - ' + d['title'])
#  .to_csv(
#     base_path + "videos_sorted.csv", index=False, sep=';'
# ))

In [18]:
videos_aux = pd.DataFrame()


videos_aux["Channel Id"] = np.arange(1, len(videos) + 1)
videos_aux[" Channel Name"] = videos["channel_name"]
videos_aux[" Video Id"] = np.arange(1, len(videos) + 1)
videos_aux[" VideoUrl"] = np.arange(1, len(videos) + 1)
videos_aux[" Title"] = videos["title_canal"]
videos_aux[" Published Date"] = videos["date"]
videos_aux[" Views"] = videos["views"]
videos_aux[" Likes"] = videos["likes"]
videos_aux[" Dislikes"] = np.arange(1, len(videos) + 1)
videos_aux[" Comments"] = videos["comments_amount"]
videos_aux[" Engagement %"] = np.arange(1, len(videos) + 1)

In [19]:
videos_aux.to_csv(base_path + "RecentUploads.csv", index=False)

In [20]:
top_videos.head()

,title,views,likes,date,comments_amount,channel_name,category,title_canal
0,Cropwise Protector | Pré-plantio: monitorament...,127000,24,2020-11-04,Transcrição,Syngenta Digital Brasil,TOP,Syngenta Digital Brasil - Cropwise Protector |...
1,Syngenta Digital | Chegou a evolução da agricu...,114000,92,2020-06-15,2,Syngenta Digital Brasil,TOP,Syngenta Digital Brasil - Syngenta Digital | C...
2,#WeConnectAg: Tecnologia na agricultura é apos...,58000,4,2020-02-10,Transcrição,Syngenta Digital Brasil,TOP,Syngenta Digital Brasil - #WeConnectAg: Tecnol...
3,#WeConnectAg: estamos vivendo a quarta revoluç...,54000,2,2020-02-03,1,Syngenta Digital Brasil,TOP,Syngenta Digital Brasil - #WeConnectAg: estamo...
4,#WeConnectAg: Mudanças climáticas pedem nova a...,0,2,2020-02-05,1,Syngenta Digital Brasil,TOP,Syngenta Digital Brasil - #WeConnectAg: Mudanç...


In [21]:
videos_aux = pd.DataFrame()


videos_aux["Channel Id"] = np.arange(1, len(videos) + 1)
videos_aux[" Channel Name"] = top_videos["channel_name"]
videos_aux[" Video Id"] = np.arange(1, len(videos) + 1)
videos_aux[" VideoUrl"] = np.arange(1, len(videos) + 1)
videos_aux[" Title"] = top_videos["title_canal"]
videos_aux[" Published Date"] = top_videos["date"]
videos_aux[" Views"] = top_videos["views"]
videos_aux[" Likes"] = top_videos["likes"]
videos_aux[" Dislikes"] = np.arange(1, len(videos) + 1)
videos_aux[" Comments"] = top_videos["comments_amount"]
videos_aux[" Engagement %"] = np.arange(1, len(videos) + 1)

In [22]:
videos_aux.to_csv(base_path + "TopUploads.csv", index=False)

last 30 days

In [23]:
# read channel IDs

with open("canais.txt", "r") as f:
    canais = f.readlines()


canais = [c.strip() for c in canais]
# take only the string after the last slash
canais = [c.split("/")[-1] for c in canais]

# print(canais)

base_url = "https://socialblade.com/youtube/channel/"

last_30days = pd.read_parquet(f'data/{canais[0].split("@")[-1]}/last_30_days.parquet')


def fix_views(view: pd.Series) -> pd.Series:
    view = view.str.replace("+", "").str.replace(",", "")
    # print(view)

    view = view.apply(lambda x: float(x.replace("K", "")) * 1000 if "K" in x else x)
    view = view.apply(
        lambda x: float(x.replace("M", "")) * 1000000 if "M" in str(x) else x
    )

    return view


last_30days = last_30days.assign(
    views=lambda x: fix_views(x["views"]), subs=lambda x: fix_views(x["subs"])
)

# Replace '---' with NaN in the 'subs' column
last_30days["subs"] = last_30days["subs"].replace("--", np.nan)

In [24]:
last_30days = last_30days.astype({"views": "float64", "subs": "float64"})

In [25]:
last_30_days_aux = pd.DataFrame()


last_30_days_aux["Channel Id"] = np.arange(5)
last_30_days_aux[" Channel Name"] = df_lifetime["channel_title"]
last_30_days_aux["  Views Last 30 Days"] = last_30days["views"]
last_30_days_aux[" Views per Day"] = last_30_days_aux["  Views Last 30 Days"] / 30
last_30_days_aux[" Total Subscribers Last 30 Days"] = last_30days["subs"]
last_30_days_aux[" Subscribers per Day"] = (
    last_30_days_aux[" Total Subscribers Last 30 Days"] / 30
)
last_30_days_aux[" Total Uploads Last 30 Days"] = [
    "ATUALIZAR",
    "ATUALIZAR",
    "ATUALIZAR",
    "ATUALIZAR",
    "ATUALIZAR",
]
last_30_days_aux[" Uploads per Week"] = np.arange(5)
last_30_days_aux[""] = np.arange(5)

In [26]:
last_30_days_aux

,Channel Id,Channel Name,Views Last 30 Days,Views per Day,Total Subscribers Last 30 Days,Subscribers per Day,Total Uploads Last 30 Days,Uploads per Week,
0,0,Syngenta Digital Brasil,0.0,0.000000,0.0,0.000000,ATUALIZAR,0,0
1,1,Farmbox,654.0,21.800000,2.0,0.066667,ATUALIZAR,1,1
2,2,xarvio Digital Farming Solutions,1677.0,55.900000,10.0,0.333333,ATUALIZAR,2,2
3,3,FieldView TV,96811.0,3227.033333,100.0,3.333333,ATUALIZAR,3,3
4,4,Aegro,45370.0,1512.333333,70.0,2.333333,ATUALIZAR,4,4


In [27]:
last_30_days_aux.to_csv(base_path + "Last30Days.csv", index=False, decimal=",")

In [28]:
# last_30days.to_csv(f'data/{canais[0].split("@")[-1]}/last_30days.csv', index=False)